In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
print(pd.unique(fish['Species']))  #어떤 종류의 생선이 있는지 Spicies 열에서 unique사용해 추출

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [3]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()  #to_numpy()메서드로 넘파일 배열로 변환

In [4]:
print(fish_input[:5])  #fish_input에 5개의 특성이 잘 저장되었는지 확인

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [6]:
fish_target = fish['Species'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler  # 훈련세트와 테스트 세트 표준화 전처리
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### k-최근접 이웃 분류를 사용해 확률 예측

In [11]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


#### 타깃 데이터를 만들 때 fish['Species'] 를 사용해 만들었기 때문에 훈련세트와 테스트 세트의 타깃 데이터에도 7개의 생선 종류가 들어가 있음
> 타깃 데이터에 2개 이상의 클래스가 들어간 경우를 **다중분류** 라고 함

In [13]:
print(kn.classes_) # 정렬된 타깃값은 classes 속성에 저장되어 있음

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


#### 테스트 세트에 있는 처음 5개 샘플의 타깃값을 예측

In [14]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [15]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])  #클래스별 확률값 반환
print(np.round(proba, decimals=4))  #round는 기본적으로 소수점 첫째자리에서 반올림을 하는데 decimal 매개변수로 유리잘 소수점 자릿수 지정

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


#### predict_proba() 메서드의 출력 순서는 classes_의 속성과 같음.

In [16]:
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


#### 3개의 최근접 이웃 만을 사용하기 때문에 가능한 확률이 0, 1/3, 2/3, 3/3 이 전부이다. 때문에 가능한 확률이 매우 적음
> **로지스틱 회귀** 사용

## 로지스틱 회귀
- 선형 방정식을 사용한 분류 알고리즘. 
- 선형회귀와 달리 시그모이드 함수나 소프트맥스 함수를 사용하여 클래스 확률을 출력  
$z = a*(Weight) + b*(length) + c*(Diagonal) + d*(Height) + e*(Width) + f$
- a,b,c,d,e = 가중치 혹은 계수